In [ ]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from src.const import paths, datasets, networks
from src.parameters import metrics
from src.datasets.extract_part_of_dataset import DatasetPartExtractor
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.results.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.results.model_export import clear_exported_models, get_best_model_from_directory
from src.results.result_plot import *
from src.keras_model.vgg16_network import VGG16Network

Using TensorFlow backend.


# Setup basic params

In [3]:
network_name = networks.vgg16
network = VGG16Network()
fit_epochs = 50
dataset = datasets.catvsdog
dataset_percentage = 5

network.print_config()

Loss: <function categorical_crossentropy at 0x14a7ec9e0>
Metrics: ['accuracy', <function precision at 0x14a95bc20>, <function recall at 0x14a95bcb0>]
Image size: (224, 224)
Grayscale: False
Batch size: 32


# Extract given percentage of source dataset

In [4]:
DatasetPartExtractor().extract(dataset, dataset_percentage)

Checking if dataset exists in /Users/igor/Python/CNN-Architecture-Comparison/data/cat-vs-dog_5/
Dataset cat-vs-dog already exists with 5% rate


# Setup dataset generators for train/test/validate sets

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)


Loading train set for cat-vs-dog_5...
Found 876 images belonging to 2 classes.
Loading test set for cat-vs-dog_5...
Found 250 images belonging to 2 classes.
Loading validation set for cat-vs-dog_5...
Found 126 images belonging to 2 classes.


# Compile model of network

In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

# Setup fitting callbacks

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
clear_exported_models(checkpoint_path)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_loss}}.h5',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [best_callback, time_callback]

# Perform model fitting

In [ ]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
28/28 [==============================] - 336s 12s/step - loss: 0.9177 - accuracy: 0.4817 - precision: 0.1875 - recall: 0.3591 - val_loss: 0.6960 - val_accuracy: 0.5000 - val_precision: 0.4982 - val_recall: 1.0000

Epoch 00001: val_loss improved from inf to 0.69598, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/vgg16/cat-vs-dog/01-0.6959773302078247.h5
Epoch 2/50
 1/28 [>.............................] - ETA: 2:20 - loss: 0.6904 - accuracy: 0.5833 - precision: 0.5833 - recall: 1.0000

# Retrieve best model found during fitting

In [ ]:
best_saved_model_path = get_best_model_from_directory(checkpoint_path)
best_saved_model = keras.models.load_model(best_saved_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)

# Evaluate best saved model

In [ ]:
validation_result = best_saved_model.evaluate_generator(
    generator=dataset_generators.validate
)

# Results

### Epoch CSV

In [ ]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

### Epoch plots

In [ ]:
val_metrics_png_file_path = f'{paths.results_location}{network_name}/{dataset}-plot-val-metrics.png'
val_loss_png_file_path = f'{paths.results_location}{network_name}/{dataset}-plot-val-loss.png'
metrics_png_file_path = f'{paths.results_location}{network_name}/{dataset}-plot-metrics.png'
loss_png_file_path = f'{paths.results_location}{network_name}/{dataset}-plot-loss.png'
loss_comparison_png_file_path = f'{paths.results_location}{network_name}/{dataset}-plot-loss-comparison.png'

plot_learning_history_val_metrics(val_metrics_png_file_path, network_name, dataset, fit_result)
plot_learning_history_val_loss(val_loss_png_file_path, network_name, dataset, fit_result)
plot_learning_history_metrics(metrics_png_file_path, network_name, dataset, fit_result)
plot_learning_history_loss(loss_png_file_path, network_name, dataset, fit_result)
plot_loss_comparison(loss_comparison_png_file_path, network_name, dataset, fit_result)

### Validation result CSV

In [ ]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)